In [41]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import itertools
from collections import Counter
from networkx.drawing.nx_agraph import graphviz_layout
from skbio.stats.composition import ilr
from skbio.stats.composition import clr
from skbio.stats.composition import multiplicative_replacement
import seaborn as sns
from matplotlib import rcParams
sns.set()
sns.set(font_scale=1.5)

In [42]:
# Bacerial phylum->strain lineages
BacterialLineages= pd.read_csv("./DataFiles/Dataframes/myGenomes.lineage.csv")
BacTaxLineageDF = BacterialLineages.set_index('taxid').copy()
BacACCLineageDF = BacterialLineages.set_index('accession').copy()

In [43]:
BacterialLineages

,accession,taxid,superkingdom,phylum,class,order,family,genus,species,strain
0,NZ_CM000441.1,455631,Bacteria,Firmicutes,Clostridia,Clostridiales,Peptostreptococcaceae,Clostridioides,Clostridioides difficile,Clostridioides difficile QCD-66c26
1,NZ_CM000714.1,526973,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae,Bacillus,Bacillus cereus,Bacillus cereus m1293
2,NZ_CM000742.1,526997,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae,Bacillus,Bacillus mycoides,Bacillus mycoides DSM 2048
3,NZ_CM000741.1,526994,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae,Bacillus,Bacillus cereus,Bacillus cereus AH1273
4,NC_013791.2,398511,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae,Bacillus,Bacillus pseudofirmus,Bacillus pseudofirmus OF4
5,NC_007164.1,306537,Bacteria,Actinobacteria,Actinobacteria,Corynebacteriales,Corynebacteriaceae,Corynebacterium,Corynebacterium jeikeium,Corynebacterium jeikeium K411
6,NC_002162.1,273119,Bacteria,Tenericutes,Mollicutes,Mycoplasmatales,Mycoplasmataceae,Ureaplasma,Ureaplasma parvum,Ureaplasma parvum serovar 3
7,NC_004088.1,187410,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Yersiniaceae,Yersinia,Yersinia pestis,Yersinia pestis KIM10+
8,NC_002655.2,155864,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia,Escherichia coli,Escherichia coli O157:H7
9,NC_002620.2,243161,Bacteria,Chlamydiae,Chlamydiia,Chlamydiales,Chlamydiaceae,Chlamydia,Chlamydia muridarum,Chlamydia muridarum str. Nigg


In [44]:
import itertools
ACCdic = {}
DirtyDic = {}
accessionList = []
allStrainNames = []
accTaxDict = {"Bacterial_Strain":[], "Accession":[], "TaxID":[], "Species":[], "Genus":[], 'Family':[], 'Order':[], 'Class':[], 'Phylum':[]}
with open("./DataFiles/Dataframes/fastaToName.txt") as fastName:
    for line1,line2,line3 in itertools.zip_longest(*[fastName]*3):
        if '_PLASMID' in line2:
            accTaxDict['Bacterial_Strain'].append(str(line2.split("org:")[1].replace("_PLASMID","").strip()))
            accTaxDict['Accession'].append(line2.split(">")[1].split("|")[0].strip())
            accTaxDict['TaxID'].append(line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip())
            try:
                accTaxDict['Species'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['species'])
                accTaxDict['Genus'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['genus'])
                accTaxDict['Family'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['family'])
                accTaxDict['Order'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['order'])
                accTaxDict['Class'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['class'])
                accTaxDict['Phylum'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['phylum'])
            except:
                try:
                    accTaxDict['Species'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['species'])
                    accTaxDict['Genus'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['genus'])
                    accTaxDict['Family'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['family'])
                    accTaxDict['Order'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['order'])
                    accTaxDict['Class'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['class'])
                    accTaxDict['Phylum'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['phylum'])
                except:
                    print("CANT FIND: "+str(line2.split("org:")[1].strip()))
        else:
            accTaxDict['Bacterial_Strain'].append(str(line2.split("org:")[1].strip()))
            accTaxDict['Accession'].append(line2.split(">")[1].split("|")[0].strip())
            accTaxDict['TaxID'].append(line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip())
            try:
                accTaxDict['Species'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['species'])
                accTaxDict['Genus'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['genus'])
                accTaxDict['Family'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['family'])
                accTaxDict['Order'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['order'])
                accTaxDict['Class'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['class'])
                accTaxDict['Phylum'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['phylum'])
            except:
                try:
                    accTaxDict['Species'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['species'])
                    accTaxDict['Genus'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['genus'])
                    accTaxDict['Family'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['family'])
                    accTaxDict['Order'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['order'])
                    accTaxDict['Class'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['class'])
                    accTaxDict['Phylum'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['phylum'])
                except:
                    print("CANT FIND: "+str(line2.split("org:")[1].strip()))
           
        
        if '_PLASMID' in line2:
            ACCdic[line1.split("==>")[1].split("<==")[0].split(".fasta")[0].strip()] = line2.split("org:")[1].split("_PLASMID")[0].replace("*","X").replace("#","").replace(",","").replace("'","").replace(";","").replace("(","").replace(")","").strip()
            DirtyDic[line1.split("==>")[1].split("<==")[0].split(".fasta")[0].strip()]= line2.split("org:")[1].strip()
        else:
            ACCdic[line1.split("==>")[1].split("<==")[0].split(".fasta")[0].strip()] = line2.split("org:")[1].replace(",","").replace("*","X").replace("#","").replace("'","").replace(";","").replace("(","").replace(")","").strip()
            DirtyDic[line1.split("==>")[1].split("<==")[0].split(".fasta")[0].strip()]= line2.split("org:")[1].strip()
            
del ACCdic['GCF_000162015.1_ASM16201v1_genomic']
del ACCdic['GCF_001879645.1_ASM187964v1_genomic']

In [45]:
accTaxDictDF1 = pd.DataFrame(data=accTaxDict)
accTaxDictDF1.set_index("Bacterial_Strain", inplace=True)
for row in accTaxDictDF1.index:
    try:
        accTaxDictDF1.at[row, 'Species'] = str(accTaxDictDF1.at[row, 'Species']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
        accTaxDictDF1.at[row, 'Genus'] = str(accTaxDictDF1.at[row, 'Genus']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
        accTaxDictDF1.at[row, 'Family'] = str(accTaxDictDF1.at[row, 'Family']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
        accTaxDictDF1.at[row, 'Order'] = str(accTaxDictDF1.at[row, 'Order']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
        accTaxDictDF1.at[row, 'Class'] = str(accTaxDictDF1.at[row, 'Class']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
        accTaxDictDF1.at[row, 'Phylum'] = str(accTaxDictDF1.at[row, 'Phylum']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
    except:
        continue
        
accTaxDictDF2 = accTaxDictDF1.reset_index().copy()
for row in accTaxDictDF2.index:
    if accTaxDictDF2.at[row,'TaxID'] == str(475375) :
        accTaxDictDF2.drop(index=row, inplace=True)
        
accTaxDictDF3 = accTaxDictDF2.set_index("Bacterial_Strain").copy()
accTaxDictDF3.head()

,Accession,TaxID,Species,Genus,Family,Order,Class,Phylum
Bacterial_Strain,,,,,,,,
Clostridioides_difficile_QCD66c26,NZ_CM000441.1,455631,Clostridioides_difficile,Clostridioides,Peptostreptococcaceae,Clostridiales,Clostridia,Firmicutes
Bacillus_cereus_m1293_strainm1293,NZ_CM000714.1,526973,Bacillus_cereus,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes
Bacillus_mycoides_DSM_2048_strainDSM_2048,NZ_CM000742.1,526997,Bacillus_mycoides,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes
Bacillus_cereus_AH1273_strainAH1273,NZ_CM000741.1,526994,Bacillus_cereus,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes
Bacillus_pseudofirmus_OF4_strainOF4,NC_013791.2,398511,Bacillus_pseudofirmus,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes


In [46]:
accTaxDictDF = accTaxDictDF3.copy()
for row in accTaxDictDF.index:
    if accTaxDictDF.at[row,'Phylum'] == 'nan' or accTaxDictDF.at[row,'Phylum'] == np.nan:
        
        accTaxDictDF.at[row,'Phylum'] = "Unclassified_Bacteria"
        accTaxDictDF.at[row,'Class'] = "Unclassified_Bacteria"
        accTaxDictDF.at[row,'Order'] = "Unclassified_Bacteria"
        accTaxDictDF.at[row,'Family'] = "Unclassified_Bacteria"
        accTaxDictDF.at[row,'Genus'] = "Unclassified_Bacteria"
        
    elif accTaxDictDF.at[row,'Class'] == 'nan' or accTaxDictDF.at[row,'Class'] == np.nan:
        
        accTaxDictDF.at[row,'Class'] = "Unclassified_"+str(accTaxDictDF.at[row,'Phylum'])
        accTaxDictDF.at[row,'Order'] = "Unclassified_"+str(accTaxDictDF.at[row,'Phylum'])
        accTaxDictDF.at[row,'Family'] = "Unclassified_"+str(accTaxDictDF.at[row,'Phylum'])
        accTaxDictDF.at[row,'Genus'] = "Unclassified_"+str(accTaxDictDF.at[row,'Phylum'])

    
    elif accTaxDictDF.at[row,'Order'] == 'nan' or accTaxDictDF.at[row,'Order'] == np.nan:
        
        accTaxDictDF.at[row,'Order'] = "Unclassified_"+str(accTaxDictDF.at[row,'Class'])
        accTaxDictDF.at[row,'Family'] = "Unclassified_"+str(accTaxDictDF.at[row,'Class'])
        accTaxDictDF.at[row,'Genus'] = "Unclassified_"+str(accTaxDictDF.at[row,'Class'])
    
    elif accTaxDictDF.at[row,'Family'] == 'nan' or accTaxDictDF.at[row,'Family'] == np.nan:
        
        accTaxDictDF.at[row,'Family'] = "Unclassified_"+str(accTaxDictDF.at[row,'Order'])
        accTaxDictDF.at[row,'Genus'] = "Unclassified_"+str(accTaxDictDF.at[row,'Order'])
        
        
    elif accTaxDictDF.at[row,'Genus'] == 'nan' or accTaxDictDF.at[row,'Genus'] == np.nan:
        
        accTaxDictDF.at[row,'Genus'] = "Unclassified_"+str(accTaxDictDF.at[row,'Family'])
        
    else:
        continue

## Load Small Sample Information DataFrame

In [47]:
allCohortInfoDF = pd.read_csv("./DataFiles/CohortInformation/SampleMappingInformation.csv").set_index("SampleName")
allCohortInfoDF.head()

,Cohort,Total_Reads,Mapped,Fraction_Mapped
SampleName,,,,
CSMA8M9R,nonIBD,19884444.0,9350717.0,0.470253
MSM6J2RU,nonIBD,19874995.0,10663018.0,0.536504
MSM79H6N,nonIBD,17635326.0,9182872.0,0.520709
MSM9VZNH,nonIBD,17498519.0,13125443.0,0.750089
MSM9VZF7,nonIBD,17406321.0,9450732.0,0.542948


## LOAD ORIGINAL STRAIN ABUNDANCE DF

In [48]:
def pullGoodSamples(df, cohortname, totalMapped):
    
    #Will pull only the good samples
    df2 = df.copy()
    df3 = df2.loc[~df2.index.duplicated(keep='first')].copy()
    sampleNameList = list(allCohortInfoDF[(allCohortInfoDF['Mapped']>totalMapped) & (allCohortInfoDF['Cohort'] == cohortname)].copy().index)
    df4= df3.loc[df3.index.intersection(sampleNameList)]
    df4.index.name = 'SampleName'
    
    #Will make sure that any duplicated strains are added together before continuing
    df4.rename(columns={"Faecalibacterium_prausnitzii_A2165_strainA2165":"Faecalibacterium_prausnitzii_strainA2165"}, inplace=True)
    df5 = df4.groupby(lambda x:x, axis=1).sum().copy()

    return(df5)

In [49]:
americaODF1 = pd.read_csv("./DataFiles/PlasmidRecalc/American_plasmid-free.tsv", sep="\t").set_index("SampleName")
metaHitODF2 = pd.read_csv("./DataFiles/PlasmidRecalc/European_plasmid-free.tsv", sep="\t").set_index("SampleName")
indianODF2 = pd.read_csv("./DataFiles/PlasmidRecalc/Indian_plasmid-free.tsv", sep="\t").set_index("SampleName")
japaneseDF = pd.read_csv("./DataFiles/PlasmidRecalc/Japanese_plasmid-free.tsv", sep="\t").set_index("SampleName")

In [50]:
americanDF = pullGoodSamples(americaODF1, 'American', 250000)
#Suspected contamination of Milkweed Yellows in this sample so it gets dropped
americanDF.drop(index=['SRS011452'], inplace=True)

indianDF = pullGoodSamples(indianODF2, 'Indian', 250000)
japaneseDF = pullGoodSamples(japaneseDF, 'Japanese', 250000)
europeanDF = pullGoodSamples(metaHitODF2, 'European', 250000)

## Clean up strain Dataframes, Turn to species, filter

In [51]:
def filterNoise(df):
    df3 = df.copy()
    
    # Quickly check if the entire column is bad
    lowMax = []
    for column in df3.columns:
        if df3[column].max() < 0.00001:
            lowMax.append(column)
    #Drop bad columns
    df4 = df3.drop(columns=lowMax).copy()
    
    #Create new dataframe and check noise levels
    thresh=0.00001
    df5 = df4.apply(lambda x: x.where(x > thresh, 0), axis=0).copy()
                
    return(df5)

In [52]:
def strainToSpecies(df):
    cleandf = df.copy()
    
    # Convert strain names into species names
    tempDictionary = { k:v for (k,v) in zip(accTaxDictDF.index, accTaxDictDF['Species'])}
    cleandf.rename(columns=tempDictionary,inplace=True)
    
    df3 = cleandf.groupby(lambda x:x, axis=1).sum().copy()
    
    return(df3)

In [53]:
def strainToSpeciesG(df):
    cleandf = df.copy()
    
    # Convert strain names into species names
    tempDictionary = { k:v for (k,v) in zip(accTaxDictDF.index, accTaxDictDF['Genus'])}
    cleandf.rename(columns=tempDictionary,inplace=True)
                  
    df3 = cleandf.groupby(lambda x:x, axis=1).sum().copy()
    return(df3)

In [54]:
def StrainToPhylum(df):
    cleandf = df.copy()
    
    # Convert Strain names into Phylum names
    tempDictionary = { k:v for (k,v) in zip(accTaxDictDF.index, accTaxDictDF['Phylum'])}
    cleandf.rename(columns=tempDictionary,inplace=True)
    
    
    df3 = cleandf.groupby(lambda x:x, axis=1).sum().copy()
    return(df3)

## Now Filter DataFrames

In [55]:
#Japanese DATAFRAME
japaDF1 = japaneseDF.copy()
japaDF2 = filterNoise(japaDF1)

japaDF3 = strainToSpecies(japaDF2)
japaDF4 = strainToSpeciesG(japaDF2)
japanesePhylum = StrainToPhylum(japaDF2)

In [56]:
#American
clean1 = americanDF.copy()
clean2 = filterNoise(clean1)

clean3 = strainToSpecies(clean2)
clean4 = strainToSpeciesG(clean2)
AmericanPhylum = StrainToPhylum(clean2)

In [57]:
#Indian
clean11 = indianDF.copy()
clean21 = filterNoise(clean11)

clean31 = strainToSpecies(clean21)
clean41 = strainToSpeciesG(clean21)
IndianPhylum = StrainToPhylum(clean21)

In [58]:
#European
clean12 = europeanDF.copy()
clean22 = filterNoise(clean12)

clean32 = strainToSpecies(clean22)
clean42 = strainToSpeciesG(clean22)
EuropeanPhylum = StrainToPhylum(clean22)

## Export Strain Dataframes

In [59]:
#American
clean2.to_csv("./DataFiles/Dataframes/strainDataframes/masterAmerican_allStrains.csv")
#Indian
clean21.to_csv("./DataFiles/Dataframes/strainDataframes/masterIndian_allStrains.csv")
#JAPANESE
japaDF2.to_csv("./DataFiles/Dataframes/strainDataframes/masterJapanese_allStrains.csv")
#EUROPEAN
clean22.to_csv("./DataFiles/Dataframes/strainDataframes/masterEuropean_allStrains.csv")

## Export Species Dataframes

In [60]:
#American
clean3.to_csv("./DataFiles/Dataframes/speciesDataframes/masterAmerican_allSpecies.csv")
#Indian
clean31.to_csv("./DataFiles/Dataframes/speciesDataframes/masterIndian_allSpecies.csv")
#JAPANESE
japaDF3.to_csv("./DataFiles/Dataframes/speciesDataframes/masterJapanese_allSpecies.csv")
#EUROPEAN
clean32.to_csv("./DataFiles/Dataframes/speciesDataframes/masterEuropean_allSpecies.csv")

## Export Genus Dataframes

In [61]:
#American
clean4.to_csv("./DataFiles/Dataframes/genusDataframes/masterAmerican_allGenera.csv")
#Indian
clean41.to_csv("./DataFiles/Dataframes/genusDataframes/masterIndian_allGenera.csv")
#JAPANESE
japaDF4.to_csv("./DataFiles/Dataframes/genusDataframes/masterJapanese_allGenera.csv")
#EUROPEAN
clean42.to_csv("./DataFiles/Dataframes/genusDataframes/masterEuropean_allGenera.csv")

## Prevalence Filters

In [65]:
def prevalenceFilter(df):
    dfo = df.copy()
    dfo.replace(0.0, np.nan, inplace=True)
    dfo.dropna(axis=1, how='any', thresh=round(len(dfo.index)*.9), inplace=True)
    dfo.fillna(0.0, inplace=True)
    dfo2 = dfo.div(dfo.sum(axis=1), axis=0).copy()            
    return(dfo2)

In [66]:
def prevalenceFilter100(df):
    dfo = df.copy()
    dfo.replace(0.0, np.nan, inplace=True)
    dfo.dropna(axis=1, how='any', thresh=round(len(dfo.index)), inplace=True)
    dfo.fillna(0.0, inplace=True)
    dfo2 = dfo.div(dfo.sum(axis=1), axis=0).copy()            
    return(dfo2)

## Start to pull the Prevalence Groups and collaps to Species/Genus

In [63]:
########################################## STRAIN 90 ############################################################
#This will only keep strains found in 90% of samples (90% Prevalence)
AmericanStrains90 = prevalenceFilter(clean2)
IndianStrains90 = prevalenceFilter(clean21)
EuropeanStrains90 = prevalenceFilter(clean22)
JapaneseStrains90 = prevalenceFilter(japaDF2)

#American
AmericanStrains90.to_csv("./DataFiles/Dataframes/strainDataframes/PrevFiltered/masterAmerican_90%Prev_Strains.csv")
#Indian
IndianStrains90.to_csv("./DataFiles/Dataframes/strainDataframes/PrevFiltered/masterIndian_90%Prev_Strains.csv")
#JAPANESE
JapaneseStrains90.to_csv("./DataFiles/Dataframes/strainDataframes/PrevFiltered/masterJapanese_90%Prev_Strains.csv")
#EUROPEAN
EuropeanStrains90.to_csv("./DataFiles/Dataframes/strainDataframes/PrevFiltered/masterEuropean_90%Prev_Strains.csv")

In [67]:
################################################# STRAIN 100 ######################################################
#This will only keep strains found in every sample (100% Prevalence)
AmericanStrains100 = prevalenceFilter100(clean2)
IndianStrains100 = prevalenceFilter100(clean21)
EuropeanStrains100 = prevalenceFilter100(clean22)
JapaneseStrains100 = prevalenceFilter100(japaDF2)

#American
AmericanStrains100.to_csv("./DataFiles/Dataframes/strainDataframes/PrevFiltered/masterAmerican_100%Prev_Strains.csv")
#Indian
IndianStrains100.to_csv("./DataFiles/Dataframes/strainDataframes/PrevFiltered//masterIndian_100%Prev_Strains.csv")
#JAPANESE
JapaneseStrains100.to_csv("./DataFiles/Dataframes/strainDataframes/PrevFiltered/masterJapanese_100%Prev_Strains.csv")
#EUROPEAN
EuropeanStrains100.to_csv("./DataFiles/Dataframes/strainDataframes/PrevFiltered/masterEuropean_100%Prev_Strains.csv")

In [68]:
#################################################### SPECIES 90 #################################################
AmericanSpecies90 = prevalenceFilter(clean3)
IndianSpecies90 = prevalenceFilter(clean31)
EuropeanSpecies90 = prevalenceFilter(clean32)
JapaneseSpecies90 = prevalenceFilter(japaDF3)


#American
AmericanSpecies90.to_csv("./DataFiles/Dataframes/speciesDataframes/PrevFiltered/masterAmerican_90%Prev_Species.csv")
#Indian
IndianSpecies90.to_csv("./DataFiles/Dataframes/speciesDataframes/PrevFiltered/masterIndian_90%Prev_Species.csv")
#JAPANESE
JapaneseSpecies90.to_csv("./DataFiles/Dataframes/speciesDataframes/PrevFiltered/masterJapanese_90%Prev_Species.csv")
#EUROPEAN
EuropeanSpecies90.to_csv("./DataFiles/Dataframes/speciesDataframes/PrevFiltered/masterEuropean_90%Prev_Species.csv")

In [69]:
#################################################### SPECIES 100 #################################################
AmericanSpecies100 = prevalenceFilter100(clean3)
IndianSpecies100 = prevalenceFilter100(clean31)
EuropeanSpecies100 = prevalenceFilter100(clean32)
JapaneseSpecies100 = prevalenceFilter100(japaDF3)

#American
AmericanSpecies100.to_csv("./DataFiles/Dataframes/speciesDataframes/PrevFiltered/masterAmerican_100%Prev_Species.csv")
#Indian
IndianSpecies100.to_csv("./DataFiles/Dataframes/speciesDataframes/PrevFiltered/masterIndian_100%Prev_Species.csv")
#JAPANESE
JapaneseSpecies100.to_csv("./DataFiles/Dataframes/speciesDataframes/PrevFiltered/masterJapanese_100%Prev_Species.csv")
#EUROPEAN
EuropeanSpecies100.to_csv("./DataFiles/Dataframes/speciesDataframes/PrevFiltered/masterEuropean_100%Prev_Species.csv")